In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

### Training Data

In [2]:
datadir = 'C:/Users/LightSpeed/Light Speed/Deep Learning/Brain Tumor Classification MRI/Augmented Training'
categories = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
train_images = []
train_labels = []
for category in categories:
    path = os.path.join(datadir,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        img_array = cv2.resize(img_array,(100,100) )
        train_images.append(img_array)
        train_labels.append(categories.index(category))

In [3]:
len(train_images),len(train_labels)

(33242, 33242)

In [4]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
train_images.shape[1:], train_labels.shape

((100, 100, 3), (33242,))

In [5]:
train_images = train_images/255.0

### Testing Data

In [6]:
datadir = os.path.normpath('C:/Users/LightSpeed/Light Speed/Deep Learning/Brain Tumor Classification MRI/Testing')
categories = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
test_images = []
test_labels = []
for category in categories:
    path = os.path.join(datadir,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        img_array = cv2.resize(img_array,(100,100) )
        test_images.append(img_array)
        test_labels.append(categories.index(category))

In [7]:
len(test_images),len(test_labels)

(394, 394)

In [8]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_images.shape[1:], test_labels.shape

((100, 100, 3), (394,))

In [9]:
test_images = test_images/255.0

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.python.keras.layers import Conv2D,MaxPooling2D
from tensorflow.python.keras import regularizers

from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers

In [12]:
weight_decay = 1e-4
model = Sequential()

#1st Convolutional Layer
model.add(Conv2D(32,(3,3),padding='same',kernel_regularizer=regularizers.l2(weight_decay) ,input_shape=train_images.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#2nd Convolutional Layer
model.add(Conv2D(64,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#3rd Convolutional Layer
model.add(Conv2D(128,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#4th Convolutional Layer
model.add(Conv2D(256,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#5th Convolutional Layer
model.add(Conv2D(512,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#6th Convolutional Layer
model.add(Conv2D(1024,(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#Fully connected layer
model.add(Flatten())
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(4,activation='softmax'))

In [13]:
sgd = keras.optimizers.SGD(0.1)

model.compile(optimizer=sgd,
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [14]:
model.fit(train_images,train_labels,
         epochs=100,
         validation_data=(test_images,test_labels))

Epoch 1/100
1039/1039 [==============================] - 62s 59ms/step - loss: 1.3482 - accuracy: 0.5880 - val_loss: 2.5717 - val_accuracy: 0.2563
Epoch 2/100
1039/1039 [==============================] - 56s 54ms/step - loss: 0.9676 - accuracy: 0.6989 - val_loss: 2.0416 - val_accuracy: 0.3959
Epoch 3/100
1039/1039 [==============================] - 57s 55ms/step - loss: 0.7997 - accuracy: 0.7758 - val_loss: 1.8207 - val_accuracy: 0.4695
Epoch 4/100
1039/1039 [==============================] - 57s 55ms/step - loss: 0.6880 - accuracy: 0.8219 - val_loss: 2.6344 - val_accuracy: 0.3934
Epoch 5/100
1039/1039 [==============================] - 58s 55ms/step - loss: 0.6106 - accuracy: 0.8511 - val_loss: 2.9108 - val_accuracy: 0.4924
Epoch 6/100
1039/1039 [==============================] - 58s 55ms/step - loss: 0.5626 - accuracy: 0.8740 - val_loss: 2.2826 - val_accuracy: 0.5609
Epoch 7/100
1039/1039 [==============================] - 58s 56ms/step - loss: 0.5137 - accuracy: 0.8922 - val_loss: 2

In [15]:
yprob_train = model.predict(train_images)
ypred_train = [np.argmax(i) for i in yprob_train]

yprob_test = model.predict(test_images)
ypred_test = [np.argmax(i) for i in yprob_test]

In [16]:
from sklearn.metrics import confusion_matrix

conf_mx_train = confusion_matrix(train_labels, ypred_train)
print(conf_mx_train)
print('*************************************************')
conf_mx_test = confusion_matrix(test_labels, ypred_test)
print(conf_mx_test)

[[9068    1    0   56]
 [  15 8885    5  113]
 [   1    1 5968   57]
 [  12    0    0 9060]]
*************************************************
[[ 26  44  28   2]
 [  0 113   2   0]
 [  1   0 102   2]
 [  0   0   6  68]]


### Tuning The model

In [12]:
sgd = keras.optimizers.SGD(0.1)
def build_model_augmented(hp):
    try:
        weight_decay = 1e-4
        model = Sequential()
        model.add(Conv2D(32,(3,3),padding='same',kernel_regularizer=regularizers.l2(weight_decay) ,input_shape=train_images.shape[1:]))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.4))

        for i in range(5,hp.Int('Number_Of_Conv_Layers',min_value=6,max_value=10)):
            model.add(Conv2D((2**i),(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
            model.add(Activation('relu'))
            model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Dropout(hp.Choice('Dropout'+str(i),[0.1,0.2,0.3,0.4,0.5])))


        model.add(Flatten())
        
        for i in range(hp.Int('Number_Of_Hidden_Layers',min_value=1,max_value=10)):
            model.add(Dense(hp.Int('Number_Of_Hidden_Nuerons'+str(i),min_value=500,max_value=10000,step=500),activation='relu'))
            model.add(Dropout(0.4))
        
        model.add(Dense(4,activation='softmax'))
        model.compile(optimizer=sgd,
                 loss = 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
        return model
    except:
        weight_decay = 1e-4
        model = Sequential()
        model.add(Conv2D(32,(3,3),padding='same',kernel_regularizer=regularizers.l2(weight_decay) ,input_shape=train_images.shape[1:]))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.4))

        for i in range(5,hp.Int('Number_Of_Conv_Layers',min_value=6,max_value=12)):
            model.add(Conv2D((2**i),(3,3), kernel_regularizer=regularizers.l2(weight_decay) , padding='same'))
            model.add(Activation('relu'))
            model.add(BatchNormalization())
            #model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Dropout(hp.Choice('Dropout'+str(i),[0.1,0.2,0.3,0.4,0.5])))


        model.add(Flatten())
        for i in range(hp.Int('Number_Of_Hidden_Layers',min_value=1,max_value=10)):
            model.add(Dense(hp.Int('Number_Of_Hidden_Nuerons'+str(i),min_value=500,max_value=10000,step=500),activation='relu'))
            model.add(Dropout(0.4))
        
        model.add(Dense(4,activation='softmax'))
        model.compile(optimizer=sgd,
                 loss = 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
        return model

In [13]:
tuner_aug = RandomSearch(
    build_model_augmented,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory=os.path.normpath('C:/'),
    project_name='Hyp_cnn_aug')

INFO:tensorflow:Reloading Oracle from existing project C:\Hyp_cnn_aug\oracle.json
INFO:tensorflow:Reloading Tuner from C:\Hyp_cnn_aug\tuner0.json


In [14]:
tuner_aug.search(train_images, train_labels,epochs=100,validation_data=(test_images,test_labels))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
Number_Of_Conv_...|6                 |9                 
Dropout5          |0.5               |0.5               
Number_Of_Hidde...|9                 |4                 
Number_Of_Hidde...|7500              |3500              
Dropout6          |0.1               |0.1               
Dropout7          |0.2               |0.3               
Number_Of_Hidde...|6500              |3000              
Number_Of_Hidde...|6500              |500               
Number_Of_Hidde...|5000              |1000              
Number_Of_Hidde...|9500              |3500              
Number_Of_Hidde...|7000              |4000              
Number_Of_Hidde...|8000              |1500              
Dropout8          |0.4               |0.2               
Dropout9          |0.1               |0.2               
Number_Of_Hidde...|8000              |2000              
Number_Of_Hidde...|7000              |9000              
Numb

Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-dca8b78e3b1e>", line 51, in build_model_augmented
    model.add(Dense(hp.Int('Number_Of_Hidden_Nuerons'+str(i),min_value=500,max_value=10000,step=500),activation='relu'))
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\training\tracking\base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py", line 221, in add
    output_tensor = layer(self.outputs[0])
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\base_layer.py", line 926, in __call__
    input_list)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensor

Invalid model 0/5


Traceback (most recent call last):
  File "<ipython-input-12-dca8b78e3b1e>", line 23, in build_model_augmented
    model.add(Dense(hp.Int('Number_Of_Hidden_Nuerons'+str(i),min_value=500,max_value=10000,step=500),activation='relu'))
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\training\tracking\base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py", line 221, in add
    output_tensor = layer(self.outputs[0])
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\base_layer.py", line 926, in __call__
    input_list)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "C:\Users\LightSpeed\AppData\Roami

Invalid model 1/5


...

Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-dca8b78e3b1e>", line 51, in build_model_augmented
    model.add(Dense(hp.Int('Number_Of_Hidden_Nuerons'+str(i),min_value=500,max_value=10000,step=500),activation='relu'))
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\training\tracking\base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py", line 221, in add
    output_tensor = layer(self.outputs[0])
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\base_layer.py", line 926, in __call__
    input_list)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensor

Invalid model 4/5


Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-dca8b78e3b1e>", line 51, in build_model_augmented
    model.add(Dense(hp.Int('Number_Of_Hidden_Nuerons'+str(i),min_value=500,max_value=10000,step=500),activation='relu'))
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\training\tracking\base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py", line 221, in add
    output_tensor = layer(self.outputs[0])
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\base_layer.py", line 926, in __call__
    input_list)
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensor

Invalid model 5/5


RuntimeError: Too many failed attempts to build model.

In [17]:
best_model = tuner_aug.get_best_models(1)[0]
best_model.evaluate(test_images,test_labels)

13/13 [==============================] - 0s 28ms/step - loss: 1.6572 - accuracy: 0.8071


[1.657168984413147, 0.807106614112854]

In [ ]:
512*512